In [1]:
import pandas as pd
import networkx as nx
import numpy as np
import glob

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# for notebook
%matplotlib inline

## Datasets

Ligue1_saison_2020_2021 : 1 temporada

In [2]:
# https://www.kaggle.com/datasets/lucmouneyres/ligue-1-saison-20202021?rvi=1

ligue1 = pd.read_csv('../data/Ligue1_saison_2020_2021/data_2020_2021.csv', sep=',')
df = ligue1.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   journee       380 non-null    object
 1   home          380 non-null    object
 2   away          380 non-null    object
 3   scores        380 non-null    object
 4   home_goal     380 non-null    int64 
 5   away_goal     380 non-null    int64 
 6   ht_home_goal  380 non-null    int64 
 7   ht_away_goal  380 non-null    int64 
dtypes: int64(4), object(4)
memory usage: 23.9+ KB


In [3]:
df.head(20)

,journee,home,away,scores,home_goal,away_goal,ht_home_goal,ht_away_goal
0,1. Journée,Girondins Bordeaux,FC Nantes,0:0 (0:0),0,0,0,0
1,1. Journée,Dijon FCO,Angers SCO,0:1 (0:1),0,1,0,1
2,1. Journée,Lille OSC,Stade Rennes,1:1 (1:0),1,1,1,0
3,1. Journée,AS Monaco,Stade Reims,2:2 (1:2),2,2,1,2
4,1. Journée,FC Lorient,RC Strasbourg,3:1 (0:1),3,1,0,1
5,1. Journée,Nîmes Olympique,Stade Brest,4:0 (2:0),4,0,2,0
6,1. Journée,OGC Nice,RC Lens,2:1 (1:1),2,1,1,1
7,1. Journée,Montpellier HSC,Olympique Lyon,2:1 (1:0),2,1,1,0
8,1. Journée,Paris Saint-Germain,FC Metz,1:0 (0:0),1,0,0,0
9,1. Journée,Olympique Marseille,AS Saint-Étienne,0:2 (0:1),0,2,0,1


In [4]:
df.tail(20)

,journee,home,away,scores,home_goal,away_goal,ht_home_goal,ht_away_goal
360,37. Journée,Girondins Bordeaux,RC Lens,3:0 (1:0),3,0,1,0
361,37. Journée,Dijon FCO,FC Nantes,0:4 (0:2),0,4,0,2
362,37. Journée,Lille OSC,AS Saint-Étienne,0:0 (0:0),0,0,0,0
363,37. Journée,FC Lorient,FC Metz,2:1 (1:0),2,1,1,0
364,37. Journée,Olympique Marseille,Angers SCO,3:2 (1:0),3,2,1,0
365,37. Journée,AS Monaco,Stade Rennes,2:1 (2:0),2,1,2,0
366,37. Journée,Montpellier HSC,Stade Brest,0:0 (0:0),0,0,0,0
367,37. Journée,OGC Nice,RC Strasbourg,0:2 (0:1),0,2,0,1
368,37. Journée,Nîmes Olympique,Olympique Lyon,2:5 (1:3),2,5,1,3
369,37. Journée,Paris Saint-Germain,Stade Reims,4:0 (2:0),4,0,2,0


In [5]:
df.columns

Index(['journee', 'home', 'away', 'scores', 'home_goal', 'away_goal',
       'ht_home_goal', 'ht_away_goal'],
      dtype='object')

In [6]:
#df = df[df['journee'] == ]
df = df[['journee', 'home', 'away', 'home_goal', 'away_goal']] 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   journee    380 non-null    object
 1   home       380 non-null    object
 2   away       380 non-null    object
 3   home_goal  380 non-null    int64 
 4   away_goal  380 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 15.0+ KB


In [7]:
df.rename(columns = {
    'journee':'year', 
    'home':'home', 
    'away':'away', 
    'home_goal':'home_goals', 
    'away_goal':'away_goals'
    }, inplace = True) 

In [8]:
df = df[['year','home','away','home_goals','away_goals']]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   year        380 non-null    object
 1   home        380 non-null    object
 2   away        380 non-null    object
 3   home_goals  380 non-null    int64 
 4   away_goals  380 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 15.0+ KB


In [9]:
df = df.dropna(how='any',axis=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   year        380 non-null    object
 1   home        380 non-null    object
 2   away        380 non-null    object
 3   home_goals  380 non-null    int64 
 4   away_goals  380 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 15.0+ KB


In [10]:
print(sorted(df['year'].unique()))
print("#temporadas = ", len(df['year'].unique()))

['1. Journée', '10. Journée', '11. Journée', '12. Journée', '13. Journée', '14. Journée', '15. Journée', '16. Journée', '17. Journée', '18. Journée', '19. Journée', '2. Journée', '20. Journée', '21. Journée', '22. Journée', '23. Journée', '24. Journée', '25. Journée', '26. Journée', '27. Journée', '28. Journée', '29. Journée', '3. Journée', '30. Journée', '31. Journée', '32. Journée', '33. Journée', '34. Journée', '35. Journée', '36. Journée', '37. Journée', '38. Journée', '4. Journée', '5. Journée', '6. Journée', '7. Journée', '8. Journée', '9. Journée']
#temporadas =  38


In [11]:
#temp = ['2017-18', '2018-19', '2019-20', '2020-21', '2021-22', '2022-23']
#df = df.loc[df['year'].isin(temp)].reset_index()

In [12]:
vhome = list(df['home'].unique())
print(len(vhome))

vaway = list(df['away'].unique())
print(len(vaway))

vertice = set(vhome) | set(vaway)
print(len(vertice))
print(list(vertice))

20
20
20
['FC Nantes', 'Paris Saint-Germain', 'Dijon FCO', 'Montpellier HSC', 'Stade Reims', 'FC Metz', 'Girondins Bordeaux', 'RC Lens', 'Stade Rennes', 'Stade Brest', 'Olympique Lyon', 'Olympique Marseille', 'AS Monaco', 'Nîmes Olympique', 'Angers SCO', 'FC Lorient', 'RC Strasbourg', 'AS Saint-Étienne', 'Lille OSC', 'OGC Nice']


In [15]:
df['home_win'] = 0.0
df['away_win'] = 0.0
df['draw'] = 0.0

df.loc[df['home_goals'] > df['away_goals'], 'home_win'] = 1.0
df.loc[df['home_goals'] < df['away_goals'], 'away_win'] = 1.0
df.loc[df['home_goals'] == df['away_goals'], 'draw'] = 1.0

df.head()

,year,home,away,home_goals,away_goals,home_win,away_win,draw
0,1. Journée,Girondins Bordeaux,FC Nantes,0,0,0.0,0.0,1.0
1,1. Journée,Dijon FCO,Angers SCO,0,1,0.0,1.0,0.0
2,1. Journée,Lille OSC,Stade Rennes,1,1,0.0,0.0,1.0
3,1. Journée,AS Monaco,Stade Reims,2,2,0.0,0.0,1.0
4,1. Journée,FC Lorient,RC Strasbourg,3,1,1.0,0.0,0.0


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   year        380 non-null    object 
 1   home        380 non-null    object 
 2   away        380 non-null    object 
 3   home_goals  380 non-null    int64  
 4   away_goals  380 non-null    int64  
 5   home_win    380 non-null    float64
 6   away_win    380 non-null    float64
 7   draw        380 non-null    float64
dtypes: float64(3), int64(2), object(3)
memory usage: 23.9+ KB


In [17]:
df2 = df.groupby(
    ['home', 'away'],
    as_index=False,
    sort=False).agg({'home_goals':'sum', 'away_goals':'sum', 'home_win':'sum', 'away_win':'sum', 'draw' : 'sum'})

df2.head(20)

,home,away,home_goals,away_goals,home_win,away_win,draw
0,Girondins Bordeaux,FC Nantes,0,0,0.0,0.0,1.0
1,Dijon FCO,Angers SCO,0,1,0.0,1.0,0.0
2,Lille OSC,Stade Rennes,1,1,0.0,0.0,1.0
3,AS Monaco,Stade Reims,2,2,0.0,0.0,1.0
4,FC Lorient,RC Strasbourg,3,1,1.0,0.0,0.0
5,Nîmes Olympique,Stade Brest,4,0,1.0,0.0,0.0
6,OGC Nice,RC Lens,2,1,1.0,0.0,0.0
7,Montpellier HSC,Olympique Lyon,2,1,1.0,0.0,0.0
8,Paris Saint-Germain,FC Metz,1,0,1.0,0.0,0.0
9,Olympique Marseille,AS Saint-Étienne,0,2,0.0,1.0,0.0


In [18]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   home        380 non-null    object 
 1   away        380 non-null    object 
 2   home_goals  380 non-null    int64  
 3   away_goals  380 non-null    int64  
 4   home_win    380 non-null    float64
 5   away_win    380 non-null    float64
 6   draw        380 non-null    float64
dtypes: float64(3), int64(2), object(2)
memory usage: 20.9+ KB


In [21]:
lista = []
for keys, infos in df2.iterrows():
    time1 = infos.home
    time2 = infos.away
    t = df2.loc[(df2['home'] == time2) & (df2['away'] == time1)][['home','away']]
    
    if len(t) == 0:
        lista.append(time1)
        print(time2,"x",time1)

print(len(lista))

0


In [22]:
time1 = "FC Nantes"
time2 = "Montpellier HSC"

t = df2.loc[(df2['home'] == time1) & (df2['away'] == time2)][['home','away','home_goals','away_goals','home_win', 'away_win', 'draw']]
print(t.values)

t = df2.loc[(df2['home'] == time2) & (df2['away'] == time1)][['home','away','home_goals','away_goals','home_win', 'away_win', 'draw']]
print(t.values)

[['FC Nantes' 'Montpellier HSC' 1 2 0.0 1.0 0.0]]
[['Montpellier HSC' 'FC Nantes' 1 1 0.0 0.0 1.0]]


In [23]:
df2.head(20)

,home,away,home_goals,away_goals,home_win,away_win,draw
0,Girondins Bordeaux,FC Nantes,0,0,0.0,0.0,1.0
1,Dijon FCO,Angers SCO,0,1,0.0,1.0,0.0
2,Lille OSC,Stade Rennes,1,1,0.0,0.0,1.0
3,AS Monaco,Stade Reims,2,2,0.0,0.0,1.0
4,FC Lorient,RC Strasbourg,3,1,1.0,0.0,0.0
5,Nîmes Olympique,Stade Brest,4,0,1.0,0.0,0.0
6,OGC Nice,RC Lens,2,1,1.0,0.0,0.0
7,Montpellier HSC,Olympique Lyon,2,1,1.0,0.0,0.0
8,Paris Saint-Germain,FC Metz,1,0,1.0,0.0,0.0
9,Olympique Marseille,AS Saint-Étienne,0,2,0.0,1.0,0.0


In [24]:
df2.tail(20)

,home,away,home_goals,away_goals,home_win,away_win,draw
360,Girondins Bordeaux,RC Lens,3,0,1.0,0.0,0.0
361,Dijon FCO,FC Nantes,0,4,0.0,1.0,0.0
362,Lille OSC,AS Saint-Étienne,0,0,0.0,0.0,1.0
363,FC Lorient,FC Metz,2,1,1.0,0.0,0.0
364,Olympique Marseille,Angers SCO,3,2,1.0,0.0,0.0
365,AS Monaco,Stade Rennes,2,1,1.0,0.0,0.0
366,Montpellier HSC,Stade Brest,0,0,0.0,0.0,1.0
367,OGC Nice,RC Strasbourg,0,2,0.0,1.0,0.0
368,Nîmes Olympique,Olympique Lyon,2,5,0.0,1.0,0.0
369,Paris Saint-Germain,Stade Reims,4,0,1.0,0.0,0.0


In [25]:
df2["weight1"] = 0.0
df2["weight2"] = 0.0

In [26]:
for keys, infos in df2.iterrows():
    time1 = infos.home
    time2 = infos.away

    t1 =  df2.loc[(df2['home'] == time1) & (df2['away'] == time2)][['away_goals','away_win','draw']].values
    t2 =  df2.loc[(df2['away'] == time1) & (df2['home'] == time2)][['home_goals','home_win','draw']].values

    g1 = p1 = 0
    for i1 in t1:
        g1 = i1[0]
        p1 = 3*i1[1] + 1*i1[2]

    g2 = p2 = 0
    for i2 in t2:
        g2 = i2[0]
        p2 = 3*i2[1] + 1*i2[2]

    df2.loc[keys,'weight1'] = g1 + g2
    df2.loc[keys,'weight2'] = p1 + p2

In [27]:
df2.head(20)

,home,away,home_goals,away_goals,home_win,away_win,draw,weight1,weight2
0,Girondins Bordeaux,FC Nantes,0,0,0.0,0.0,1.0,3.0,4.0
1,Dijon FCO,Angers SCO,0,1,0.0,1.0,0.0,4.0,6.0
2,Lille OSC,Stade Rennes,1,1,0.0,0.0,1.0,1.0,1.0
3,AS Monaco,Stade Reims,2,2,0.0,0.0,1.0,2.0,1.0
4,FC Lorient,RC Strasbourg,3,1,1.0,0.0,0.0,2.0,1.0
5,Nîmes Olympique,Stade Brest,4,0,1.0,0.0,0.0,1.0,1.0
6,OGC Nice,RC Lens,2,1,1.0,0.0,0.0,1.0,0.0
7,Montpellier HSC,Olympique Lyon,2,1,1.0,0.0,0.0,2.0,0.0
8,Paris Saint-Germain,FC Metz,1,0,1.0,0.0,0.0,1.0,0.0
9,Olympique Marseille,AS Saint-Étienne,0,2,0.0,1.0,0.0,3.0,6.0


In [28]:
df2.tail(20)

,home,away,home_goals,away_goals,home_win,away_win,draw,weight1,weight2
360,Girondins Bordeaux,RC Lens,3,0,1.0,0.0,0.0,2.0,3.0
361,Dijon FCO,FC Nantes,0,4,0.0,1.0,0.0,5.0,4.0
362,Lille OSC,AS Saint-Étienne,0,0,0.0,0.0,1.0,1.0,2.0
363,FC Lorient,FC Metz,2,1,1.0,0.0,0.0,4.0,3.0
364,Olympique Marseille,Angers SCO,3,2,1.0,0.0,0.0,4.0,3.0
365,AS Monaco,Stade Rennes,2,1,1.0,0.0,0.0,3.0,3.0
366,Montpellier HSC,Stade Brest,0,0,0.0,0.0,1.0,2.0,2.0
367,OGC Nice,RC Strasbourg,0,2,0.0,1.0,0.0,2.0,3.0
368,Nîmes Olympique,Olympique Lyon,2,5,0.0,1.0,0.0,5.0,4.0
369,Paris Saint-Germain,Stade Reims,4,0,1.0,0.0,0.0,0.0,0.0


In [29]:
for keys, infos in df2.iterrows():
    time1 = infos.home
    time2 = infos.away

    t2 = df2.loc[(df2['home'] == time2) & (df2['away'] == time1)]
    
    if len(t2) == 0:
        t1 =  df2.loc[(df2['home'] == time1) & (df2['away'] == time2)][['home_goals','home_win','draw']].values

        w1 = w2 = 0
        for i1 in t1:
            w1 = i1[0]
            w2 = 3*i1[1] + 1*i1[2]

        new = [time2, time1,np.nan,np.nan,0,0,0,w1,w2]

        new = pd.DataFrame(columns=df2.columns, data=[new])
        
        df2 = pd.concat([df2, new], axis=0)

In [30]:
df2.head(20)

,home,away,home_goals,away_goals,home_win,away_win,draw,weight1,weight2
0,Girondins Bordeaux,FC Nantes,0,0,0.0,0.0,1.0,3.0,4.0
1,Dijon FCO,Angers SCO,0,1,0.0,1.0,0.0,4.0,6.0
2,Lille OSC,Stade Rennes,1,1,0.0,0.0,1.0,1.0,1.0
3,AS Monaco,Stade Reims,2,2,0.0,0.0,1.0,2.0,1.0
4,FC Lorient,RC Strasbourg,3,1,1.0,0.0,0.0,2.0,1.0
5,Nîmes Olympique,Stade Brest,4,0,1.0,0.0,0.0,1.0,1.0
6,OGC Nice,RC Lens,2,1,1.0,0.0,0.0,1.0,0.0
7,Montpellier HSC,Olympique Lyon,2,1,1.0,0.0,0.0,2.0,0.0
8,Paris Saint-Germain,FC Metz,1,0,1.0,0.0,0.0,1.0,0.0
9,Olympique Marseille,AS Saint-Étienne,0,2,0.0,1.0,0.0,3.0,6.0


In [31]:
df2.tail(20)

,home,away,home_goals,away_goals,home_win,away_win,draw,weight1,weight2
360,Girondins Bordeaux,RC Lens,3,0,1.0,0.0,0.0,2.0,3.0
361,Dijon FCO,FC Nantes,0,4,0.0,1.0,0.0,5.0,4.0
362,Lille OSC,AS Saint-Étienne,0,0,0.0,0.0,1.0,1.0,2.0
363,FC Lorient,FC Metz,2,1,1.0,0.0,0.0,4.0,3.0
364,Olympique Marseille,Angers SCO,3,2,1.0,0.0,0.0,4.0,3.0
365,AS Monaco,Stade Rennes,2,1,1.0,0.0,0.0,3.0,3.0
366,Montpellier HSC,Stade Brest,0,0,0.0,0.0,1.0,2.0,2.0
367,OGC Nice,RC Strasbourg,0,2,0.0,1.0,0.0,2.0,3.0
368,Nîmes Olympique,Olympique Lyon,2,5,0.0,1.0,0.0,5.0,4.0
369,Paris Saint-Germain,Stade Reims,4,0,1.0,0.0,0.0,0.0,0.0


In [32]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   home        380 non-null    object 
 1   away        380 non-null    object 
 2   home_goals  380 non-null    int64  
 3   away_goals  380 non-null    int64  
 4   home_win    380 non-null    float64
 5   away_win    380 non-null    float64
 6   draw        380 non-null    float64
 7   weight1     380 non-null    float64
 8   weight2     380 non-null    float64
dtypes: float64(5), int64(2), object(2)
memory usage: 26.8+ KB


In [33]:
df3 = df2[['home','away','weight1','weight2']]

df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   home     380 non-null    object 
 1   away     380 non-null    object 
 2   weight1  380 non-null    float64
 3   weight2  380 non-null    float64
dtypes: float64(2), object(2)
memory usage: 12.0+ KB


In [34]:
df3.head(20)

,home,away,weight1,weight2
0,Girondins Bordeaux,FC Nantes,3.0,4.0
1,Dijon FCO,Angers SCO,4.0,6.0
2,Lille OSC,Stade Rennes,1.0,1.0
3,AS Monaco,Stade Reims,2.0,1.0
4,FC Lorient,RC Strasbourg,2.0,1.0
5,Nîmes Olympique,Stade Brest,1.0,1.0
6,OGC Nice,RC Lens,1.0,0.0
7,Montpellier HSC,Olympique Lyon,2.0,0.0
8,Paris Saint-Germain,FC Metz,1.0,0.0
9,Olympique Marseille,AS Saint-Étienne,3.0,6.0


In [35]:
df3.tail(20)

,home,away,weight1,weight2
360,Girondins Bordeaux,RC Lens,2.0,3.0
361,Dijon FCO,FC Nantes,5.0,4.0
362,Lille OSC,AS Saint-Étienne,1.0,2.0
363,FC Lorient,FC Metz,4.0,3.0
364,Olympique Marseille,Angers SCO,4.0,3.0
365,AS Monaco,Stade Rennes,3.0,3.0
366,Montpellier HSC,Stade Brest,2.0,2.0
367,OGC Nice,RC Strasbourg,2.0,3.0
368,Nîmes Olympique,Olympique Lyon,5.0,4.0
369,Paris Saint-Germain,Stade Reims,0.0,0.0


In [37]:
df3.to_csv('../data/work1/ligue1_1_38.csv', index=False, sep=';')  